In [ ]:
import pandas as pd
import multiprocessing as mp
import numpy as np
from datetime import datetime
import os
import errno

Specify Test , Output, Files

In [ ]:
test = ["D:\datascience\hw2-rdf-2016_test.txt"]
#test = ["D:\homework2\hw2-rdf-2016_test\hw2-rdf-2016_test.txt"]
files = []
for i in range(1,3):
    files.append("D:\datascience\hw2-rdf-2016_{}.dat".format(str(i)))
    #files.append("D:\homework2\data\hw2-rdf-2016_1\hw2-rdf-2016_{}.dat".format(str(i)))
output_file = "D:\datascience\output_final.csv"
#output_file = "D:\homework2\output\output_final.csv"    

Function that does the work

In [ ]:
def dothejob(input,temporarydf):  
    for f in input:
        with open(f,'r') as f:
            print("Now working on file {} ".format(str(f).split("'")[1]))
            #Acquire a TextFileReader to read in chunks
            reader = pd.read_table(f, header=None,chunksize=50000) 
            for read_chunk in reader:    
                read_chunk=read_chunk[0].str.split(" ", expand=True)
                read_chunk['EDGES'] = read_chunk.stack().str.contains(r'^\"|<|_:').astype(np.uint8).sum(level=0)-1
                read_chunk[read_chunk['EDGES'] >= 3] = 3
                read_chunk[read_chunk['EDGES'] <=2] =2 
                read_chunk.rename(columns={0:'SUBJECT'}, inplace=True)
                #append the so-far read chunk into temporarydf (need to assign to temporarydf otherwise append is lost)
                temporarydf = temporarydf.append(read_chunk[['SUBJECT','EDGES']], ignore_index=True)
                print(temporarydf)
                temporarydf = temporarydf.groupby('SUBJECT', as_index=False).sum()
                del read_chunk   ### delete the read chunk and iterate loop for next 50000 lines
                writetofile(temporarydf)   #### write to csv
                
def writetofile(somedf):
    #if not os.path.exists(output_file):
    #    open(output_file, 'w').close() 
    with open(output_file,'a+') as final:  #File Object final is the .csv with intermediate values
        somedf.to_csv(final)

        
def main(inputargs):
    #### create an empty dataframe that is passed to the function 
    df_temp = pd.DataFrame(columns=['SUBJECT','EDGES'])
    df_temp = df_temp.fillna(0) # with 0s rather than NaNs
    #### created a temp dataframe
    #  RUN
    t1 = datetime.now()
    dothejob(inputargs,df_temp)    # WILL RUN FOR FIRST FILE ONLY
    print("finished after {} ".format(datetime.now() - t1))

In [ ]:
main(files)